Copyright (c) 2025 Quan-feng WU <wuquanfeng@ihep.ac.cn>

This software is released under the MIT License.
https://opensource.org/licenses/MIT

# Preliminaries

In [ ]:
using ArbNumerics, DifferentialEquations, Integrals
using NaturalUnits

using CairoMakie, LaTeXStrings

In [ ]:
set_theme!(theme_latexfonts())

include("tool_script-directory.jl")
include("tool_script-geomspace.jl")

In [ ]:
EU = GeV
NU = NaturalUnit(EU)

# arb_float = identity
arb_float = x -> ArbFloat(x; bits=1000)

aᵢ = arb_float(1)

In [ ]:
save_plot_flag = false

In [ ]:
function prepare_parameters(mϕ, λϕφφ;
    Hᵢ_OVER_mϕ = arb_float(1),
    max_du = arb_float(1e100),
    flag_1to3 = false
)
    Hᵢ = Hᵢ_OVER_mϕ * mϕ
    nϕ_a³ᵢ = 3 * NU.M_Pl^2 * Hᵢ^2 * aᵢ^3 / mϕ

    return (
        mϕ = mϕ, λϕφφ = λϕφφ,
        Hᵢ = Hᵢ,
        nϕ_a³ᵢ = nϕ_a³ᵢ,
        max_du = max_du,
        flag_1to3 = flag_1to3,
    )
end

# Evolutions

In [ ]:
function dnϕ_a³_OVER_da_FOR_ϕ_TO_φφ(a, H, nϕ_a³, parameters)
    mϕ = parameters.mϕ
    λϕφφ = parameters.λϕφφ

    return -(λϕφφ^2 / (32 * π * mϕ * a * H)) * exp(π * λϕφφ^2 * nϕ_a³ / (mϕ^4 * a^3 * H)) * nϕ_a³
end

function dnϕ_a³_OVER_da_FOR_ϕ_TO_φφh(a, H, nϕ_a³, parameters)
    mϕ = parameters.mϕ
    λϕφφ = parameters.λϕφφ
    nϕ_a³ᵢ = parameters.nϕ_a³ᵢ

    δₕ = H / mϕ
    if δₕ ≥ 1 / 2
        return zero(nϕ_a³)
    end

    nφ_a³ = (nϕ_a³ᵢ - nϕ_a³) / 2
    nϕ = nϕ_a³ / a^3
    nφ = nφ_a³ / a^3

    Γ3ϕ¹ = -(mϕ * nϕ / (64 * (2π)^3)) * (λϕφφ / NU.M_Pl)^2 * (2 * log(2 * δₕ) + 3 - 8 * δₕ + 4 * δₕ^2)
    Γ3ϕ² = (nϕ * nφ / (2 * (2π) * mϕ^2)) * (λϕφφ / NU.M_Pl)^2 * (a / aᵢ)^2 * (
        δₕ ≤ (1 - aᵢ / a) / 2 ?
            (aᵢ / a) * ((1 - aᵢ / (2a)) / (1 - aᵢ / a)) + log(1 - aᵢ / a) :
            inv(4δₕ) + log(2δₕ) - δₕ
    )
    Γ3ϕ³ = (π * nϕ * nφ^2 / (4 * mϕ^5)) * (λϕφφ / NU.M_Pl)^2 * (a / aᵢ)^4 * ((1 - 2aᵢ / a) / (1 - aᵢ / a))^2 *
        ((a / aᵢ ≤ 2 && δₕ ≤ min(1 / 2, 1 - aᵢ / a)) ? 1 : 0)

    Γ3ϕ = Γ3ϕ¹ + Γ3ϕ² + Γ3ϕ³
    
    # open("tmp.log", "a+") do io
    #     println(io, "$(η)")
    #     println(io, "  δₕ = $(δₕ)")
    #     println(io, "  v = $(Γ3ϕ / (H * nϕ_a³))")
    # end

    return -Γ3ϕ / (H * a)
end

In [ ]:
function evolution_system!(du_list, u_list, parameters, η)
    nϕ_a³_in_EU, ρₜₒₜ_a⁴_in_EU = exp.(u_list)
    a = exp(η)

    nϕ_a³ = EU(nϕ_a³_in_EU, 3)
    ρₜₒₜ_a⁴ = EU(ρₜₒₜ_a⁴_in_EU, 4)

    mϕ = parameters.mϕ
    λϕφφ = parameters.λϕφφ
    nϕ_a³ᵢ = parameters.nϕ_a³ᵢ

    ρϕ_a³ = min(mϕ * nϕ_a³, ρₜₒₜ_a⁴ / a)
    nϕ_a³ = ρϕ_a³ / mϕ
    # nφ_a³ = (nϕ_a³ᵢ - nϕ_a³) / 2

    ρₜₒₜ = ρₜₒₜ_a⁴ / a^4
    H = sqrt(ρₜₒₜ / (3 * NU.M_Pl^2))

    dnϕ_a³_OVER_da = dnϕ_a³_OVER_da_FOR_ϕ_TO_φφ(a, H, nϕ_a³, parameters)
    dρₜₒₜ_a⁴_OVER_da = ρϕ_a³

    parameters.flag_1to3 && (dnϕ_a³_OVER_da += dnϕ_a³_OVER_da_FOR_ϕ_TO_φφh(a, H, nϕ_a³, parameters))

    du_list[1] = dnϕ_a³_OVER_da * a / nϕ_a³
    du_list[2] = dρₜₒₜ_a⁴_OVER_da * a / ρₜₒₜ_a⁴

    # if all(isa(du, typeof(arb_float(1))) for du ∈ du_list)
    #     open("tmp.log", "a+") do io
    #         println(io, "------------------------------")
    #         println(io, "$(η)")
    #         println(io, "  $(du_list[1])")
    #         println(io, "  $(du_list[2])")
    #     end
    # end

    for (ii, du) ∈ enumerate(du_list)
        if abs(du) ≥ parameters.max_du
            du_list[ii] = parameters.max_du * sign(du)
        end
    end

    # if all(isa(du, typeof(arb_float(1))) for du ∈ du_list)
    #     open("tmp.log", "a+") do io
    #         println(io)
    #         println(io, "  $(du_list[1])")
    #         println(io, "  $(du_list[2])")
    #         println(io)
    #     end
    # end

    return du_list
end

In [ ]:
function solve_evolution_system(parameters;
    aₑ = arb_float(1e6),
    solver = Rosenbrock23()
)
    nϕ_a³ᵢ = parameters.nϕ_a³ᵢ
    ρₜₒₜ_a⁴ᵢ = mϕ * nϕ_a³ᵢ * aᵢ

    evolution_problem = ODEProblem(evolution_system!,
        [
            (log ∘ EUval)(EU, nϕ_a³ᵢ),
            (log ∘ EUval)(EU, ρₜₒₜ_a⁴ᵢ),
        ],
        (log(aᵢ), log(aₑ)),
        parameters
    )
    evolution_solution = solve(evolution_problem, solver)
    evolution_solution.retcode == ReturnCode.Success || @warn "ODE return code: $(evolution_solution.retcode)"

    return evolution_solution
end

# Playgrounds

In [ ]:
mϕ = GeV(arb_float(1e13))
λϕφφ = GeV(arb_float(8e8))
aₑ = arb_float(1e12)
parameters = prepare_parameters(mϕ, λϕφφ; max_du = arb_float(1e200), flag_1to3 = true)
rm("tmp.log"; force=true)
# evolution_solution = solve_evolution_system(parameters; aₑ = aₑ, solver = Tsit5())
# evolution_solution = solve_evolution_system(parameters; aₑ = aₑ, solver = Rosenbrock23())
# evolution_solution = solve_evolution_system(parameters; aₑ = aₑ, solver = Rodas4P())
evolution_solution = solve_evolution_system(parameters; aₑ = aₑ, solver = Rodas5P())

a_list = geomspace(aᵢ, aₑ, 10000)
nϕ_a³_list = ((v -> EU(v, 3)) ∘ exp ∘ first ∘ evolution_solution ∘ log).(a_list)
ρₜₒₜ_a⁴_list = ((v -> EU(v, 4)) ∘ exp ∘ last ∘ evolution_solution ∘ log).(a_list)
ρϕ_a³_list = min.(mϕ .* nϕ_a³_list, ρₜₒₜ_a⁴_list ./ a_list)

ρₜₒₜ_list = ρₜₒₜ_a⁴_list ./ a_list.^4
ρϕ_list = ρϕ_a³_list ./ a_list.^3
ρφ_list = ρₜₒₜ_list .- ρϕ_list

ρφ_a⁴_list = ρφ_list .* a_list.^4

H_list = sqrt.(ρₜₒₜ_list ./ (3 * NU.M_Pl^2))

dnϕ_a³_OVER_da_FOR_ϕ_TO_φφ_list =
    dnϕ_a³_OVER_da_FOR_ϕ_TO_φφ.(a_list, H_list, nϕ_a³_list, Ref(parameters))
dnϕ_a³_OVER_da_FOR_ϕ_TO_φφh_list =
    dnϕ_a³_OVER_da_FOR_ϕ_TO_φφh.(a_list, H_list, nϕ_a³_list, Ref(parameters))

figure = Figure()

axis = Axis(figure[1, 1],
    xlabel = L"a / a_I",
    ylabel = L"\left[\mathrm{GeV}^4\right]",
    limits = ((aᵢ, 1.2 * aₑ / 10), (1e50, 1e73)),
    xscale = log10, yscale = log10,
    xticks = (ticklist -> (exp10.(ticklist), [L"10^{%$ii}" for ii ∈ ticklist]))(0:100),
    yticks = (ticklist -> (exp10.(ticklist), [L"10^{%$ii}" for ii ∈ ticklist]))(0:2:100),
)
lines!(axis, a_list ./ aᵢ, EUval.(EU, ρₜₒₜ_a⁴_list), label = L"\rho_{\mathrm{tot}} a^4")
lines!(axis, a_list ./ aᵢ, EUval.(EU, ρϕ_a³_list), label = L"\rho_{\phi} a^3")
lines!(axis, a_list ./ aᵢ, EUval.(EU, ρφ_a⁴_list), label = L"\rho_{\varphi} a^4")
axislegend(axis; position = :lt)

figure

In [ ]:
mϕ = GeV(arb_float(1e13))
λϕφφ = GeV(arb_float(8e8))
aₑ = arb_float(1e12)
parameters = prepare_parameters(mϕ, λϕφφ; max_du = arb_float(1e200), flag_1to3 = true)
rm("tmp.log"; force=true)
# evolution_solution = solve_evolution_system(parameters; aₑ = aₑ, solver = Tsit5())
# evolution_solution = solve_evolution_system(parameters; aₑ = aₑ, solver = Rosenbrock23())
# evolution_solution = solve_evolution_system(parameters; aₑ = aₑ, solver = Rodas4P())
evolution_solution = solve_evolution_system(parameters; aₑ = aₑ, solver = Rodas5P())

a_list = geomspace(aᵢ, aₑ, 10000)
nϕ_a³_list = ((v -> EU(v, 3)) ∘ exp ∘ first ∘ evolution_solution ∘ log).(a_list)
ρₜₒₜ_a⁴_list = ((v -> EU(v, 4)) ∘ exp ∘ last ∘ evolution_solution ∘ log).(a_list)
ρϕ_a³_list = min.(mϕ .* nϕ_a³_list, ρₜₒₜ_a⁴_list ./ a_list)

ρₜₒₜ_list = ρₜₒₜ_a⁴_list ./ a_list.^4
ρϕ_list = ρϕ_a³_list ./ a_list.^3
ρφ_list = ρₜₒₜ_list .- ρϕ_list

ρφ_a⁴_list = ρφ_list .* a_list.^4

H_list = sqrt.(ρₜₒₜ_list ./ (3 * NU.M_Pl^2))

dnϕ_a³_OVER_da_FOR_ϕ_TO_φφ_list =
    dnϕ_a³_OVER_da_FOR_ϕ_TO_φφ.(a_list, H_list, nϕ_a³_list, Ref(parameters))
dnϕ_a³_OVER_da_FOR_ϕ_TO_φφh_list =
    dnϕ_a³_OVER_da_FOR_ϕ_TO_φφh.(a_list, H_list, nϕ_a³_list, Ref(parameters))

figure = Figure(size = (800, 1200))

axis = Axis(figure[1, 1],
    xlabel = L"a / a_I",
    ylabel = L"\left[\mathrm{GeV}^4\right]",
    limits = ((aᵢ, 10), (1e50, 1e73)),
    # limits = ((aᵢ, 1.2 * aₑ / 10), (1e50, 1e73)),
    xscale = log10, yscale = log10,
    xticks = (ticklist -> (exp10.(ticklist), [L"10^{%$ii}" for ii ∈ ticklist]))(0:100),
    yticks = (ticklist -> (exp10.(ticklist), [L"10^{%$ii}" for ii ∈ ticklist]))(0:2:100),
)
lines!(axis, a_list ./ aᵢ, EUval.(EU, ρₜₒₜ_a⁴_list), label = L"\rho_{\mathrm{tot}} a^4")
lines!(axis, a_list ./ aᵢ, EUval.(EU, ρϕ_a³_list), label = L"\rho_{\phi} a^3")
# lines!(axis, a_list ./ aᵢ, EUval.(EU, ρφ_a⁴_list), label = L"\rho_{\varphi} a^4")
axislegend(axis; position = :lt)

axis = Axis(figure[2, 1],
    xlabel = L"a / a_I",
    # ylabel = L"\mathrm{d} n_{\phi} a^3 / \mathrm{d} a \left[\mathrm{GeV}^3\right]",
    limits = ((aᵢ, 10), (1e0, 1e50)),
    # limits = ((aᵢ, 1.2 * aₑ / 10), (1e0, 1e50)),
    xscale = log10, yscale = log10,
    xticks = (ticklist -> (exp10.(ticklist), [L"10^{%$ii}" for ii ∈ ticklist]))(0:100),
    # yticks = (ticklist -> (exp10.(ticklist), [L"10^{%$ii}" for ii ∈ ticklist]))(-100:5:30),
    xminorgridvisible = true, xminorticks = IntervalsBetween(9)
)
lines!(axis, a_list ./ aᵢ, (abs ∘ EUval).(EU, dnϕ_a³_OVER_da_FOR_ϕ_TO_φφ_list))
lines!(axis, a_list ./ aᵢ, (abs ∘ EUval).(EU, dnϕ_a³_OVER_da_FOR_ϕ_TO_φφh_list))

figure